In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt; plt.style.use('classic')
import matplotlib.ticker as ticker
import seaborn as sns; sns.set()
import pandas as pd
from mgcpy.independence_tests.mgc.mgc import MGC
from mgcpy.benchmarks import simulations as sims
from mgcpy.independence_tests.dcorr import DCorr
from scipy.stats import bernoulli
import math

In [4]:
import numpy as np
np.random.seed(88889999)
import graspy
from graspy.inference import NonparametricTest
from graspy.embed import AdjacencySpectralEmbed, select_dimension

from graspy.simulations import sbm, rdpg
from graspy.utils import symmetrize, get_lcc
from graspy.plot import heatmap, pairplot
from scipy.stats import bernoulli
from mgcpy.hypothesis_tests.transforms import k_sample_transform
from tqdm import tqdm_notebook as tqdm

In [14]:
class power_test():
    def __init__(self):
        self.mc = 100
        self.alpha = 0.05

    def _gen(self, null, n):
        X1 = np.random.uniform(0.2,0.7,n).reshape(-1,1)
        X2 = np.random.uniform(0.4,0.9,n).reshape(-1,1)
        A1 = rdpg(X1,
            loops=False,
            rescale=False,
            directed=False)
        if null:
            A2 = rdpg(X1,
                loops=False,
                rescale=False,
                directed=False)
        else:
            A2 = rdpg(X2,
                loops=False,
                rescale=False,
                directed=False)
        # A1 = graspy.utils.get_lcc(A1)
        # A2 = graspy.utils.get_lcc(A2)
        self.G = A1
        self.F = A2
    
    def _embed(self):
        num_dims1 = select_dimension(self.G)[0][-1]
        num_dims2 = select_dimension(self.F)[0][-1]
        n_components = max(num_dims1, num_dims2)

        ase = AdjacencySpectralEmbed(n_components=n_components)
        self.X = ase.fit_transform(self.G)
        self.Y = ase.fit_transform(self.F)

    def power(self, alg, null, n, mc=100, alpha=0.05):
        self._gen(null=null, n=n)
        
        if alg == 'npt':
            self.alg = NonparametricTest(n_bootstraps=1)
        if alg == 'mgc':
            self.alg = MGC()
        
        test_stat_null_array = np.zeros(self.mc)
        test_stat_alt_array = np.zeros(self.mc)
        for i in range(self.mc):
            if alg == 'mgc':
                self._embed()
                X, Y = k_sample_transform(self.X, self.Y)
                X2 = np.random.permutation(X)
                test_stat_alt, _ = self.alg.test_statistic(
                    matrix_X=self.X, matrix_Y=self.Y, is_fast=True)
                test_stat_null, _ = self.alg.test_statistic(
                    matrix_X=X2, matrix_Y=Y, is_fast=True) 
            elif alg == 'npt':
                self.alg.fit(self.G, self.F)
                test_stat_alt = self.alg.sample_T_statistic_
                test_stat_null = self.alg.null_distribution_[0]
            
            test_stat_alt_array[i] = test_stat_alt
            test_stat_null_array[i] = test_stat_null

        critical_value = np.sort(test_stat_null_array)[math.ceil((1-alpha)*mc)]
        power = np.where(test_stat_alt_array > critical_value)[0].shape[0] / mc
        return power

In [16]:
import warnings
warnings.filterwarnings("ignore")

In [18]:
pt = power_test()
pt.power('mgc', False, 50)

0.0